In [1]:
#Background of character and situation

In [14]:
#Import Libraries
import pandas as pd
from pathlib import Path
import os
import hvplot.pandas
import numpy as np
from datetime import date
import seaborn as sns
import json
#import kaggle

ModuleNotFoundError: No module named 'cartopy'

#Import file from API (MUST HAVE KAGGLE INSTALLED: ("conda install kaggle")
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()


api.dataset_download_file('syuzai/perth-house-prices','all_perth_310121.csv')

In [2]:
#CSV file was unzipped from downloaded file

#Read CSV file
all_perth_data = pd.read_csv("all_perth_310121.csv")

In [3]:
#Drop NAs
all_perth_data = all_perth_data.dropna()

#Confirming all nulls have been removed
all_perth_data.isnull().sum()

ADDRESS             0
SUBURB              0
PRICE               0
BEDROOMS            0
BATHROOMS           0
GARAGE              0
LAND_AREA           0
FLOOR_AREA          0
BUILD_YEAR          0
CBD_DIST            0
NEAREST_STN         0
NEAREST_STN_DIST    0
DATE_SOLD           0
POSTCODE            0
LATITUDE            0
LONGITUDE           0
NEAREST_SCH         0
NEAREST_SCH_DIST    0
NEAREST_SCH_RANK    0
dtype: int64

In [4]:
#Remove \r from the Date_Sold column commented as the data is already saved in the data frame.
all_perth_data['DATE_SOLD'] = all_perth_data['DATE_SOLD'].str.strip('\r')
#Converting date sold column as a date time data type
all_perth_data['DATE_SOLD'] = pd.to_datetime(all_perth_data['DATE_SOLD'])

In [5]:
# Second option concatenation with another csv file
project_path_2 = Path("./residential_property_price.csv")

In [6]:
# Reading the residential_property_price data 
price_df = pd.read_csv(project_path_2)
#Filtering data points where MEDIAN HOUSE PRICE is greater than zero. 
price_df = price_df[price_df['MEDIAN HOUSE PRICE'] > 0]
#Setting the index of the data frame as SUBURB
price_df = price_df.set_index(price_df["SUBURB"])
#Displaying the first few rows of the Data frame
price_df.head()

,SUBURB,MEDIAN HOUSE PRICE,GROWTH PERCENTAGE
SUBURB,,,
Alexander Heights,Alexander Heights,490000,8.90%
Alfred Cove,Alfred Cove,921000,17.00%
Alkimos,Alkimos,452500,10.40%
Applecross,Applecross,1735000,-0.60%
Ardross,Ardross,1165000,15.90%


In [7]:
#Selecting specific columns from the data frame
suburbs_df = all_perth_data[['SUBURB', 'LATITUDE', 'LONGITUDE']].groupby(['SUBURB']).mean()
#Concatinating both the data frames.
suburbs_df_price = pd.concat([price_df, suburbs_df], axis="columns", join="inner")
#Displaying first few data points of data frame
suburbs_df_price.head()

,SUBURB,MEDIAN HOUSE PRICE,GROWTH PERCENTAGE,LATITUDE,LONGITUDE
SUBURB,,,,,
Alexander Heights,Alexander Heights,490000,8.90%,-31.830310,115.866293
Alfred Cove,Alfred Cove,921000,17.00%,-32.033908,115.816117
Alkimos,Alkimos,452500,10.40%,-31.618988,115.691004
Applecross,Applecross,1735000,-0.60%,-32.013569,115.838399
Ardross,Ardross,1165000,15.90%,-32.026691,115.838054


In [8]:
#Turning Growth Percentage to Float data type to sort correctly
suburbs_df_price['GROWTH PERCENTAGE'] = suburbs_df_price['GROWTH PERCENTAGE'].str.strip('%')
suburbs_df_price['GROWTH PERCENTAGE'] = suburbs_df_price['GROWTH PERCENTAGE'].astype(float)

In [12]:
#Full Geo map grouped by suburbs
suburbs_df_price.hvplot.points('LONGITUDE',
                                       'LATITUDE', 
                                       geo=True,
                                       color='MEDIAN HOUSE PRICE',
                                       size='MEDIAN HOUSE PRICE',
                                       scale=0.013,
                                       tiles='OSM',
                                       frame_width=700,
                                       frame_height=500,
                                       hover = True,
                                       hover_cols=['SUBURB','MEDIAN HOUSE PRICE']
                                        )

ImportError: Geographic projection support requires GeoViews and cartopy.

Client 1 has come to us andbdgnwgjoekgfpsfk,ps

In [16]:
#Input of client requirements
#Creating a new data frame df by dropping Build_year, postcode, date sold
df = all_perth_data.drop(columns=['BUILD_YEAR', 'POSTCODE', 'DATE_SOLD'])
#Requesting the clients property requirements
bedrooms = int(input('Please enter the number of Bedrooms : '))
plan_budget = int(input('Please enter your budget : '))
landsize = int(input('Please enter your desired minimum land size : '))
cbddist = int(input('Please enter the maximum distance from CBD in metres: '))
#Filtering out properties based on clients requirements 
client1 = df.loc[(all_perth_data['BEDROOMS'] >= bedrooms)
                               & (df['PRICE'] <= plan_budget)
                               & (df['CBD_DIST'] <= cbddist)
                               & (df['LAND_AREA'] >= landsize)]
#Displaying the properties which meet clients requirements
client1

Please enter the number of Bedrooms :  2
Please enter your budget :  300000
Please enter your desired minimum land size :  1
Please enter the maximum distance from CBD in metres:  2000


,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
20398,39/193 Hay Street,East Perth,258000,2,1,1.0,2425,80,1600,Claisebrook Station,1100,-31.959205,115.873203,MERCEDES COLLEGE,0.665447,15.0
23351,47 Smith Street,Highgate,284000,2,1,2.0,238,101,1700,East Perth Station,644,-31.941076,115.870885,PERTH COLLEGE,1.183227,9.0
28248,65/273 Hay Street,East Perth,290000,2,1,1.0,1882,80,1300,McIver Station,822,-31.958307,115.870264,MERCEDES COLLEGE,0.378614,15.0


In [17]:
#Visualization displaying the properties which meet clients requirements
client1.hvplot.points('LONGITUDE',
                                       'LATITUDE', 
                                       geo=True,
                                       color='ADDRESS',
                                       tiles='OSM',
                                       size= 'PRICE',
                                       scale= 0.05,
                                       frame_width=700,
                                       frame_height=500,
                                       hover = True,
                                       hover_cols=['SUBURB','PRICE', 'CBD_DIST', 'BEDROOMS']
                                        )

ImportError: Geographic projection support requires GeoViews and cartopy.

In [18]:
#Creating an empty list
temp_list = {}
#Loading the suburb which the property is located in.
temp_list = client1['SUBURB']
#Printing the median price and the growth rate of the property.
display(price_df[price_df['SUBURB'].isin(temp_list)])

,SUBURB,MEDIAN HOUSE PRICE,GROWTH PERCENTAGE
SUBURB,,,
East Perth,East Perth,1950000,77.30%
Highgate,Highgate,994000,10.50%


In [21]:
#Calculators
#Defining mortgage calculator 
def mortgagecalculator (mortgage, years, interest):
#Converting decimal input to floating point value
    interest = interest / 100
#Calculating the number of periods per year   
    nper = years*12

#Calculating the monthly intrest payment
    interest_monthly = interest/12
    numerator = interest_monthly*((1+interest_monthly)**nper)
    denominator = (1+interest_monthly)**nper-1
#Returning the result to function call and formatting the displayes numerical.
    payment = float("{0:.2f}".format(mortgage*numerator/denominator))
    return(payment)

In [22]:
#Collecting clients mortgage details.
#Requesting client the loan amount
mortgate_amount=int(input('What is the mortgage amount? '))
#Enquiring clint of the duration to repay the loan
no_of_years=int(input('For how many years? '))
#Collecting the intrest rate information from the bank
intrest_rate=int(input('What interest rate? '))
#Calling the function to provide the monthly repayment
mortgagecalculator(mortgate_amount,no_of_years,intrest_rate)

What is the mortgage amount?  3333333333
For how many years?  4
What interest rate?  2


72317078.79

In [ ]:
#Definition of stamp duty calculator
#Calculating the stamp duty based on the value of the property.
def stampdutycalculator (purchase_price):
    if purchase_price <= 120000:
        stamp_duty = purchase_price*0.019
    elif purchase_price > 120000 and purchase_price <= 150000:
        stamp_duty = 2280 + (purchase_price - 120000) * 0.0285
    elif purchase_price > 150000 and purchase_price <= 360000:
        stamp_duty = 3135 + (purchase_price - 150000) * 0.038
    elif purchase_price > 360000 and purchase_price <= 725000:
        stamp_duty = 11175 + (purchase_price - 360000) * 0.0475
    else:
        stamp_duty = 28453 + (purchase_price - 725000) * 0.0515
#Returning the calculated stamp duty to the function call   
    return stamp_duty

In [ ]:
#Entering the property value dynamically.
property_value=int(input('What is the property value? '))
#Calling the function definition to calculate the property_value
stampdutycalculator(property_value)

In [ ]:
#A new client story and parameters

In [ ]:
#New geo plot that contains house options for client B

In [ ]:
#Calculators

In [ ]:
#A couple of analysis questions including: If you had unlimited money, where would be the best value for money for an investment price per land area etc

#What are the top 3 suburbs to invest in by future growth percentage

#What are the top 3 suburbs to avoid by future growth percentage

In [ ]:
suburbs_df_price = suburbs_df_price.sort_values(['GROWTH PERCENTAGE'], ascending=False)

In [ ]:
display(suburbs_df_price.head(5), suburbs_df_price.tail(5))